In [1]:
from collections import namedtuple
from math import inf, prod
from Crypto.Util.number import inverse, GCD
from typing import List
from sympy import factorint, isprime, prime

Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return "Point("+str(p.x)+","+str(p.y)+")" if not p.isinf else "O"
Point.__str__ = pt_to_str

EllipticCurve = namedtuple("EllipticCurve", "p a b")


def ec_add(curve: EllipticCurve, p1: Point, p2: Point):
    # assert not (p1.isinf and p2.isinf), "cannot add O to O"
    if p1.isinf: return p2
    if p2.isinf: return p1
    if (p1.x==p2.x and p1.y==-p2.y): return Point(0,0,isinf=True)
    if p1.x!=p2.x:
        tangent = (p2.y - p1.y)*inverse(p2.x - p1.x, curve.p) # pow(p2.x - p1.x, curve.p-2, curve.p)
    else:
        tangent = (3*(p1.x)**2 + curve.a)*inverse(2*p1.y, curve.p) # pow(2*p1.y, curve.p-2, curve.p)
    p3x = tangent**2 - p1.x - p2.x
    p3y = tangent*(p1.x - p3x) - p1.y
    return Point(p3x%curve.p, p3y%curve.p)


def ec_mul(curve: EllipticCurve, p: Point, n: int):
    Q = p
    R = Point(0,0,True)
    while (n>0):
        if n%2==1: R = ec_add(curve, Q, R)
        Q = ec_add(curve, Q, Q)
        n = n//2
    return R

def ec_neg(curve: EllipticCurve, p: Point):
    return Point(p.x, -p.y % curve.p, p.isinf)

In [73]:
import random
from Crypto.Util.number import inverse, GCD

def func_f(E: EllipticCurve, X_i: Point, P: Point, Q: Point):
    if int(X_i.x) % 3 == 2:
        return ec_add(E, X_i, Q)
    if int(X_i.x) % 3 == 0:
        return ec_mul(E, X_i, 2)
    if int(X_i.x) % 3 == 1:
        return ec_add(E, X_i, P)
    else:
        print("[-] Something's Wrong!")
        return -1

def func_g(E: EllipticCurve, order: int, a: int, P: Point, X_i: Point):
    n = order # hardcode order here
    if int(X_i.x) % 3 == 2:
        return a
    if int(X_i.x) % 3 == 0:
        return 2*a % n
    if int(X_i.x) % 3 == 1:
        return (a + 1) % n
    else:
        print("[-] Something's Wrong!")
        return None

def func_h(E, order: int, b, P, X_i):
    n = order # hardcode order here
    if int(X_i.x) % 3 == 2:
        return (b + 1) % n
    if int(X_i.x) % 3 == 0:
        return 2*b % n
    if int(X_i.x) % 3 == 1:
        return b
    else:
        print("[-] Something's Wrong!")
        return None

def pollardrho(E, order, P, Q):
    n = order
    for j in range(10):
        a_i = random.randint(2,  order-2)
        b_i = random.randint(2,  order-2)
        a_2i = random.randint(2, order-2)
        b_2i = random.randint(2, order-2)

        X_i =  ec_add(E, ec_mul(E, P, a_i), ec_mul(E, Q, b_i))
        X_2i = ec_add(E, ec_mul(E, P, a_2i), ec_mul(E, Q, b_2i))

        i = 1
        while i <= n:
            # Single Step Calculations
            a_i = func_g(E, order, a_i, P, X_i)
            b_i = func_h(E, order, b_i, P, X_i)
            X_i = func_f(E, X_i, P, Q)

            # Double Step Calculations
            a_2i = func_g(E, order, func_g(E, order, a_2i, P, X_2i), P, func_f(E, X_2i, P, Q))
            b_2i = func_h(E, order, func_h(E, order, b_2i, P, X_2i), P, func_f(E, X_2i, P, Q))
            X_2i = func_f(E, func_f(E, X_2i, P, Q), P, Q)

            if X_i == X_2i:
                if b_i == b_2i:
                    break
                assert GCD(b_2i - b_i, n) == 1
                return ((a_i - a_2i) * inverse(b_2i - b_i, n)) % n
            else:
                i += 1
                continue

In [3]:
order_db = {
    310717010502520989590157367261876774703: 310717010502520989590206149059164677804,
    165229: 165792,
    270778799: 270781544,
    31850531: 31852392,
    9281: 9236,
    2309: 2320,
    179317983307: 179317476944,
    11207342309: 11207218780,
    4441: 4568
}

In [4]:
def subgroup_discrete_log(curve: EllipticCurve, order: int, P: Point, Q: Point):
    ret = []
    for b, e in factorint(order).items():
        acc = 0
        for i in range(e):
            result = ec_mul(curve, ec_add(curve, Q, ec_neg(curve, ec_mul(curve, P, acc))), order//(b**(i+1)))
            # print(result, ec_mul(curve, P, order//b))
            a = pollardrho(curve, b**e, result, ec_mul(curve, P, order//b))
            acc+=a*(b**i)
        ret.append(acc)
    return ret

In [5]:
def pohlig_hellman_EC(E, order, P, Q, debug=True):
    n = order
    factors = [p_i**e_i for (p_i, e_i) in factorint(n).items()]
    crt_array = []
    for p_i in factors[1:]:
        print(p_i)
        g_i = ec_mul(E, P, (n // p_i))
        h_i = ec_mul(E, Q, (n // p_i))
        x_i = pollardrho(E, p_i, g_i, h_i)
        if debug and x_i != None:
            print("[x] Found discrete logarithm %d for factor %d" % (x_i, p_i))
            crt_array += [x_i]
        elif x_i == None:
            print("[] Did not find discrete logarithm for factor %d" % p_i)
    return crt_array, factors


In [6]:
# ans: 47836431801801373761601790722388100620

In [8]:
E = EllipticCurve(7919, 1001, 75)
m = 7889

P = Point(4023, 6036)
Q = Point(4135, 3169)

E = EllipticCurve(310717010502520989590157367261876774703, 2, 3)
m = 310717010502520989590206149059164677804 # order of the curve

P = Point(179210853392303317793440285562762725654,105268671499942631758568591033409611165)
Q = Point(280810182131414898730378982766101210916,291506490768054478159835604632710368904)
# subgroup_discrete_log(E, m, P, Q)

In [83]:
E = EllipticCurve(17, 2, 2)
P = Point(5,1)
Q = ec_mul(E, P, 7)
m = 19
pollardrho(E, 19, P, Q)
# ec_mul(E, Q, (m // 7))

7

In [43]:
[ec_mul(E, ec_mul(E, P, (m // 7)), i) for i in range(1,28)]

[Point(x=15, y=8, isinf=False),
 Point(x=23, y=24, isinf=False),
 Point(x=3, y=16, isinf=False),
 Point(x=3, y=21, isinf=False),
 Point(x=23, y=13, isinf=False),
 Point(x=15, y=29, isinf=False),
 Point(x=15, y=8, isinf=False),
 Point(x=15, y=8, isinf=False),
 Point(x=23, y=24, isinf=False),
 Point(x=3, y=16, isinf=False),
 Point(x=3, y=21, isinf=False),
 Point(x=23, y=13, isinf=False),
 Point(x=15, y=29, isinf=False),
 Point(x=23, y=24, isinf=False),
 Point(x=23, y=24, isinf=False),
 Point(x=23, y=24, isinf=False),
 Point(x=3, y=16, isinf=False),
 Point(x=3, y=21, isinf=False),
 Point(x=23, y=13, isinf=False),
 Point(x=15, y=29, isinf=False),
 Point(x=3, y=21, isinf=False),
 Point(x=15, y=8, isinf=False),
 Point(x=3, y=16, isinf=False),
 Point(x=3, y=16, isinf=False),
 Point(x=3, y=21, isinf=False),
 Point(x=23, y=13, isinf=False),
 Point(x=15, y=29, isinf=False)]